In [15]:
import zipfile
from io import BytesIO
import requests
import os

from osgeo import ogr
import pandas as pd

from us import states

In [28]:
census_csv = pd.read_csv('/Users/joe/Dropbox/SFI_Census_Stuff/2010Census/2010.csv', dtype={'ID': str})

1. Download & extract all zip files
2. ??
3. make ID column
4. combine census data & shapefiles
5. 

In [20]:
def download_shape(fips, extract_path):
    """
        gz_2010_ss_150_00_500k.zip, where ss is the 2 digit state FIPS code.
    """
    shapefile_url = 'http://www2.census.gov/geo/tiger/GENZ2010/'
    shapefile_url += 'gz_2010_{}_150_00_500k.zip'.format(fips)
    
    r = requests.get(shapefile_url)
    zfile = zipfile.ZipFile(BytesIO(r.content))
    zfile.extractall(path=extract_path)
    return zfile

In [3]:
def add_id_field(file_path):
    source = ogr.Open(file_path, 1)
    layer = source.GetLayer()
    layer_defn = layer.GetLayerDefn()
    field_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]

    # Add a new field
    new_field = ogr.FieldDefn('ID', ogr.OFTString)
    layer.CreateField(new_field)

    feature = layer.GetNextFeature()
    while feature:
        feature.SetField("ID", feature.GetField("GEO_ID").split('US')[1] )
        layer.SetFeature(feature)
        feature = layer.GetNextFeature()

    # Close the Shapefile
    source.Destroy()

In [81]:
def merge_files(directory, outputMergefn, fileStartsWith='gz_'):
    fileEndsWith = '.shp'
    driverName = 'ESRI Shapefile'
    geometryType = ogr.wkbPolygon

    out_driver = ogr.GetDriverByName( driverName )
    if os.path.exists(outputMergefn):
        out_driver.DeleteDataSource(outputMergefn)
    out_ds = out_driver.CreateDataSource(outputMergefn)
    out_layer = out_ds.CreateLayer(outputMergefn, geom_type=geometryType)

    fileList = os.listdir(directory)

    create_fields = True
    for file in fileList:
        if file.startswith(fileStartsWith) and file.endswith(fileEndsWith):
            print (file)
            ds = ogr.Open(directory+file)
            lyr = ds.GetLayer()
            lyr_def = lyr.GetLayerDefn()
            
            if create_fields:
                # Only create new fields once
                # Create OLD Fields
                for i in range(lyr_def.GetFieldCount()):
                    fieldName = lyr_def.GetFieldDefn(i).GetName()              
                    fieldTypeCode =lyr_def.GetFieldDefn(i).GetType()

                    new_field = ogr.FieldDefn(fieldName, fieldTypeCode)
                    out_layer.CreateField(new_field)
                    
                # Create fields from CSV
                dtypes = {'float64':ogr.OFTReal, 'int64':ogr.OFTInteger, 'object':ogr.OFTString}
                for col in census_csv.columns:
                    #new_field = ogr.FieldDefn(col, dtypes[str(census_csv[col].dtype)])
                    new_field = ogr.FieldDefn(col, ogr.OFTString)
                    out_layer.CreateField(new_field)

                # Create ID field
                new_field = ogr.FieldDefn('ID', ogr.OFTString)
                out_layer.CreateField(new_field)

                # Create Density field
                new_field = ogr.FieldDefn('Density', ogr.OFTReal)
                out_layer.CreateField(new_field)
                
                create_fields = False

            for feat in lyr:
                feat_id = feat.GetField("GEO_ID").split('US')[1]
                pd_row = census_csv[census_csv.ID == feat_id]
                if len(pd_row) > 0:
                    out_feat = ogr.Feature(out_layer.GetLayerDefn())
                    out_feat.SetGeometry(feat.GetGeometryRef().Clone())
                    # Only get features in our dataset (i.e. in cities)

                    # Add old fields to new feature
                    old_fields = [lyr_def.GetFieldDefn(i).GetName() for i in range(lyr_def.GetFieldCount())]
                    for field in old_fields:
                        out_feat.SetField(field, feat.GetField(field))
                    # Add new ID field
                    out_feat.SetField("ID",  feat_id)
                    # Add new Density field
                    if feat.GetField("CENSUSAREA") > 0:
                        density = float(pd_row['TOTPOP'])/feat.GetField("CENSUSAREA")
                        out_feat.SetField("Density",  density)
                    else:
                        out_feat.SetField("Density",  0)
                        
                    
                    for col in pd_row.columns:
                        out_feat.SetField(str(col), str(pd_row.iloc[0].to_dict()[col]))
                
                    out_layer.CreateFeature(out_feat)
                    out_layer.SyncToDisk()
            ds.Destroy()

In [83]:
def download_merge():
    temp_path = 'tmp'
    for state in states.STATES:
        continue
        #download_shape(state.fips, temp_path)
    merge_files(temp_path + '/', 'us_bkgps_2010.shp')
download_merge()

gz_2010_01_150_00_500k.shp
gz_2010_02_150_00_500k.shp
gz_2010_04_150_00_500k.shp
gz_2010_05_150_00_500k.shp
gz_2010_06_150_00_500k.shp
gz_2010_08_150_00_500k.shp
gz_2010_09_150_00_500k.shp
gz_2010_10_150_00_500k.shp
gz_2010_11_150_00_500k.shp
gz_2010_12_150_00_500k.shp
gz_2010_13_150_00_500k.shp
gz_2010_15_150_00_500k.shp
gz_2010_16_150_00_500k.shp
gz_2010_17_150_00_500k.shp
gz_2010_18_150_00_500k.shp
gz_2010_19_150_00_500k.shp
gz_2010_20_150_00_500k.shp
gz_2010_21_150_00_500k.shp
gz_2010_22_150_00_500k.shp
gz_2010_23_150_00_500k.shp
gz_2010_24_150_00_500k.shp
gz_2010_25_150_00_500k.shp
gz_2010_26_150_00_500k.shp
gz_2010_27_150_00_500k.shp
gz_2010_28_150_00_500k.shp
gz_2010_29_150_00_500k.shp
gz_2010_30_150_00_500k.shp
gz_2010_31_150_00_500k.shp
gz_2010_32_150_00_500k.shp
gz_2010_33_150_00_500k.shp
gz_2010_34_150_00_500k.shp
gz_2010_35_150_00_500k.shp
gz_2010_36_150_00_500k.shp
gz_2010_37_150_00_500k.shp
gz_2010_38_150_00_500k.shp
gz_2010_39_150_00_500k.shp
gz_2010_40_150_00_500k.shp
g